<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)"> Predictive Analysis - Image Processing</h2>

[source](https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/)

#### Classify handwritten digits using the famous MNIST data

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/02/mnist.png)

The goal in this task is to take an image of a handwritten single digit, and determine what that digit is.  

The data for this competition were taken from the MNIST dataset. The MNIST ("Modified National Institute of Standards and Technology") dataset is a classic within the Machine Learning community that has been extensively studied.  More detail about the dataset, including Machine Learning algorithms that have been tried on it and their levels of success, can be found at http://yann.lecun.com/exdb/mnist/index.html.  

In [1]:
import os
import sys
import time
import glob
import math
import random
import pickle
import pylab
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.font_manager import FontProperties
from matplotlib import pyplot

from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.decomposition import PCA

import tensorflow as tf

%matplotlib inline
#matplotlib.rcdefaults()
#matplotlib.verbose.set_level('silent')
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from IPython.display import Image
from IPython.core.display import HTML

2022-02-11 23:00:00.366520: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-11 23:00:00.366543: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.8.0


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.python.keras import utils
from sklearn.metrics import accuracy_score

In [4]:
mnist = tf.keras.datasets.mnist

# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# let's print the shape of the dataset

In [5]:
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

X_train shape (60000, 28, 28)
y_train shape (60000,)
X_test shape (10000, 28, 28)
y_test shape (10000,)


In [6]:
# Flattening the images from the 28x28 pixels to 1D 787 pixels
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [7]:
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [9]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = tf.keras.utils.to_categorical(y_train, n_classes)
Y_test = tf.keras.utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


## ANN

In [10]:
# building a linear stack of layers with the sequential model
model = Sequential()
model.add(Dense(100, input_shape=(784,), activation='relu'))
model.add(Dense(10, activation='softmax'))

# looking at the model summary
model.summary()

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# training the model for 10 epochs
%time model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_test, Y_test))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2022-02-11 23:03:30.648971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-11 23:03:30.649409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-11 23:03:30.649476: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-02-11 23:03:30.649524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-02-11 23:03:30.649585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

469/469 [==============================] - 1s 2ms/step - loss: 0.3753 - accuracy: 0.8982 - val_loss: 0.2039 - val_accuracy: 0.9423
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1754 - accuracy: 0.9497 - val_loss: 0.1479 - val_accuracy: 0.9564
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1297 - accuracy: 0.9625 - val_loss: 0.1224 - val_accuracy: 0.9632
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1027 - accuracy: 0.9705 - val_loss: 0.1073 - val_accuracy: 0.9677
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0841 - accuracy: 0.9763 - val_loss: 0.0968 - val_accuracy: 0.9704
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0696 - accuracy: 0.9802 - val_loss: 0.0882 - val_accuracy: 0.9731
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.0596 - accuracy: 0.9832 - val_loss: 0.0830 - val_accuracy: 0.9750
Epoch 8/10
469/46

In [11]:
model.evaluate(X_test, Y_test, verbose=1)

313/313 [==============================] - 0s 754us/step - loss: 0.0808 - accuracy: 0.9750


[0.08078442513942719, 0.9750000238418579]

## CNN

In [12]:
# loading the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [13]:
# building the input vector from the 28x28 pixels
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [14]:
# normalizing the data to help with the training
X_train /= 255
X_test /= 255

In [15]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = tf.keras.utils.to_categorical(y_train, n_classes)
Y_test = tf.keras.utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

Shape before one-hot encoding:  (60000,)
Shape after one-hot encoding:  (60000, 10)


In [16]:
# building a linear stack of layers with the sequential model
model = Sequential()

model.add(Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())

model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))

# looking at the model summary
model.summary()

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


# training the model for 10 epochs
%time model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_test, Y_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 10)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6760)              0         
                                                                 
 dense_2 (Dense)             (None, 100)               676100    
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 677,210
Trainable params: 677,210
Non-trainable params: 0
________________________________________________

In [17]:
model.evaluate(X_test, Y_test, verbose=1)

313/313 [==============================] - 1s 2ms/step - loss: 0.0609 - accuracy: 0.9838


[0.06085588410496712, 0.9837999939918518]